In [20]:
import os
import numpy as np
from pathlib import Path
from pdbfixer import PDBFixer

# OpenMM Application Layer
from openmm import XmlSerializer
from openmm import app
from openmm.app import Modeller, PDBFile

# OpenMM Library Layer
from openmm import Platform, LangevinIntegrator

# OpenMM Units
from openmm import unit

# OPENFF
from openmmforcefields.generators import SystemGenerator

In [21]:
# Set input parameters
pdb_path = Path("./examples/9bf9_Mono_LAG3.cif")
output_dir = Path("TMP")
output_dir.mkdir(parents=True, exist_ok=True)

In [22]:
# Fix the protein structure
print("Preparing protein...")
fixer = PDBFixer(str(pdb_path))
fixer.removeHeterogens(keepWater=False)
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()

print("Missing residues: ", fixer.missingResidues)
print("Missing atoms: ", fixer.missingAtoms)
print("Nonstandard residues: ", fixer.nonstandardResidues)

fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.4)

Preparing protein...
Missing residues:  {}
Missing atoms:  {<Residue 43 (SER) of chain 0>: [<Atom 3 (OG) of chain 0 residue 0 (SER)>], <Residue 44 (TRP) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (TRP)>, <Atom 4 (CD1) of chain 0 residue 0 (TRP)>, <Atom 5 (NE1) of chain 0 residue 0 (TRP)>, <Atom 6 (CE2) of chain 0 residue 0 (TRP)>, <Atom 7 (CZ2) of chain 0 residue 0 (TRP)>, <Atom 8 (CH2) of chain 0 residue 0 (TRP)>, <Atom 9 (CZ3) of chain 0 residue 0 (TRP)>, <Atom 10 (CE3) of chain 0 residue 0 (TRP)>, <Atom 11 (CD2) of chain 0 residue 0 (TRP)>], <Residue 156 (ARG) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (ARG)>, <Atom 4 (CD) of chain 0 residue 0 (ARG)>, <Atom 5 (NE) of chain 0 residue 0 (ARG)>, <Atom 6 (CZ) of chain 0 residue 0 (ARG)>, <Atom 7 (NH1) of chain 0 residue 0 (ARG)>, <Atom 8 (NH2) of chain 0 residue 0 (ARG)>]}
Nonstandard residues:  []


In [23]:
# Setup forcefield
print("Setting up forcefield...")
forcefield_kwargs = {
    'constraints': app.HBonds, 
    'rigidWater': False, 
    'removeCMMotion': False,
    'hydrogenMass': 4 * unit.amu
}

# Setup system generator for APO (protein-only) system
system_generator = SystemGenerator(
    forcefields=['amber/ff14SB.xml', 'amber/tip3p_standard.xml'],
    forcefield_kwargs=forcefield_kwargs
)

# Create modeller with protein and add solvent
print("Creating system and adding solvent...")
modeller = Modeller(fixer.topology, fixer.positions)
modeller.addSolvent(system_generator.forcefield,
                    model='tip3p',
                    padding=1 * unit.nanometer,
                    ionicStrength=0.15 * unit.molar)

Setting up forcefield...
Creating system and adding solvent...


In [24]:
# Create system
system = system_generator.create_system(modeller.topology)
print(system.getNumParticles())

70066


In [25]:
with open(output_dir / "system.xml", "w") as f:
    f.write(XmlSerializer.serialize(system))

In [10]:
# Set up integrator
print("Setting up integrator...")
temperature = 300  # Kelvin
friction = 1.0     # 1/ps
timestep = 0.002   # ps
integrator = LangevinIntegrator(
    temperature * unit.kelvin,
    friction / unit.picosecond,
    timestep * unit.picoseconds
)

# Setup platform (try HIP/CUDA)
try:
    platform = Platform.getPlatformByName('HIP')
    properties = {'Precision': 'mixed', 'DeviceIndex': '0'}
except Exception:
    try:
        platform = Platform.getPlatformByName('CUDA')
        properties = {'Precision': 'mixed', 'CudaDeviceIndex': '0'}
    except Exception:
        platform = Platform.getPlatformByName('CPU')
        properties = {}

# Create simulation
print("Creating simulation...")
simulation = app.Simulation(modeller.topology, system, integrator, platform, properties)
simulation.context.setPositions(modeller.positions)

Setting up integrator...
Creating simulation...


In [ ]:
# Report system size
system_size = len(list(modeller.topology.atoms()))
print(f"System size: {system_size} atoms")

# Save the prepared system as PDB
print("Saving prepared system...")
state = simulation.context.getState(getPositions=True)
with open(os.path.join(output_dir, "prepared_system.pdb"), "w") as f:
    PDBFile.writeFile(modeller.topology, state.getPositions(), f)

# Optional: Save minimized structure
print("Minimizing system...")
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True)
energy_after_min = min_state.getPotentialEnergy()
print(f"Energy after minimization: {energy_after_min}")

with open(os.path.join(output_dir, "minimized_system.pdb"), "w") as f:
    PDBFile.writeFile(modeller.topology, min_state.getPositions(), f)

print("System preparation complete!")